In [ ]:
# Order preparation for SnP

# STATUS: Incomplete
# Run-time: 10 seconds

# Dependencies:
# /zdata/pkls/*.pkl - for pickles generated by 01_snp_scan program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 1300, clientId=0)

In [ ]:
%%time
import datetime
import pandas as pd
import numpy as np
from os import listdir
import math
import datetime
import itertools

from helper import get_dte, get_prec, grp_opts, filter_kxdte, get_onlyputs, get_hvstpricepct, get_opt_margin

#...assignments
#______________

fspath = '../data/snp/'

m_maxp = 0.015    # % of max margin allowed on net liquidity per scrip to limit positon risk
base = 0.01       # Upper or Lower base multiple for prices
expmult = 1.1    # expected multiple of price (1.05 means 5% higher)
desired_rom = 0.8 # desired rom to give the target price.

ass_limit = 40000.0   # max allowable assignment limit

min_rom = 0.3
min_pop = 0.81
min_dte = 45    # no of minimum dte days to determine ohlc filter for strikes

max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin

dow30 = list(pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average', header=0)[1].Symbol)

# from https://www.fool.com/slideshow/25-top-performing-sp-500-stocks-2018
dowtop2018 = ['AMD', 'ABMD', 'FTNT', 'TRIP', 'AAP', 'RHT', 'CMG', 'KEYS', 'ORLY', 'FOX', 'HCA', 
             'BSX', 'MCK', 'ILMN', 'NRG', 'MRK', 'LLY', 'NFLX', 'EW', 'CRM', 'AES', 'RMD', 'CHD', 'LW', 'VRSN']

topstocks = set(dow30 + dowtop2018)

#...get current positions
#________________________

#... read the account info
ac = ib.accountValues()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#...Harvest preparation
#______________________

#... read the positions
ps = ib.portfolio()

# Do harvesting only if there is something in the portfolio
if ps:
    df_p = util.df(ps)

    #...identify Stock and Option
    rights = [o.right for o in df_p.contract]
    df_p = df_p.assign(right=rights)

    df_p = df_p.assign(type=[type(x).__name__ for x in df_p.contract])

    df_p['symbol'] = [s.symbol for s in df_p.contract.values]

    # harvest option Dataframe
    df_hop = df_p[df_p.type == 'Option'].reset_index(drop=True)

    #... get the harvest as lower of discount from curve * averageCost and discount * marketPrice
    
    df_hop = df_hop.assign(averageCost=df_hop.averageCost/100) # Correct the average cost for options
    
    expiry = [d.lastTradeDateOrContractMonth for d in df_hop.contract]

    df_hop['dte'] = [get_dte(d.lastTradeDateOrContractMonth) for d in df_hop.contract]

    discount = [m for m in map(get_hvstpricepct, expiry)]
    df_hop['hvstPrice'] = pd.concat([df_hop.averageCost*discount, 
                                   df_hop.marketPrice*(1-np.array(discount))], axis=1).min(axis=1)

    df_hop.hvstPrice = np.floor(df_hop.hvstPrice/base)*base # round down to the nearest 0.05

    df_hop.loc[df_hop.hvstPrice == 0, 'hvstPrice'] = base  # make the 0s to 5 paise

    # harvest open positions with hvstPrice
    df_hop['harvestOrder'] = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) for position, hvstPrice in zip(df_hop.position, df_hop.hvstPrice)]

    # ignore data for dte < 3 days. These are as good as gone.
    df_h = df_hop[df_hop.dte > 3].reset_index(drop=True)

    hqc = ib.qualifyContracts(*df_h.contract)
    df_h = df_h.assign(qual_contract=hqc)

    # ...sowing prepration
    #_____________________

    #... find margins
    orders = [Order(action=np.where(q>0, 'SELL', 'BUY'), totalQuantity=abs(q), orderType='MKT') for q in df_p.position]

    cs = list(df_p.contract)
    qc = ib.qualifyContracts(*cs)
    df_p = df_p.assign(margin=[float(ib.whatIfOrder(c, o).initMarginChange) for c, o in zip(qc, orders)])
    df_p['lot'] = np.where(df_p.type == 'Option', df_p.position*100, df_p.position)

    #... make the blacklist
    df2 = df_p.groupby(['symbol']).sum()
    df2['max_margin'] = -max_p

    df2['avail_margin'] = df2.max_margin - df2.margin

    # determine no of units free
    df2['remqty'] = -(df2.avail_margin/df2.margin/df2.lot/100).apply(np.floor)
    df2.loc[df2.remqty <= 0, 'remqty'] = 0

    blacklist = list(df2[df2.remqty <= 0].index)

    # get remaining lots of partially filled symbols
    not_black = df2[df2.remqty > 0][['remqty']].to_dict('dict')
    remqtydict = [v for k, v in not_black.items()][0]
else:
    blacklist = []
    remqtydict = []

#...build the high-pop-roc dataframe
fs = listdir(fspath)

opts = ([f[:-8]+'_opt.pkl' for f in fs if f[-8:] == '_opt.pkl'])
ohlcs = ([f[:-8]+'_ohlc.pkl' for f in fs if f[-8:] == '_opt.pkl'])
unds = ([f[:-8]+'_und.pkl' for f in fs if f[-8:] == '_opt.pkl'])

df_opt = pd.concat([pd.read_pickle(fspath+f) for f in opts], axis=0, sort=True).reset_index(drop=True).sort_values('rom', ascending=False)
df_ohlc = pd.concat([pd.read_pickle(fspath+f).reset_index() for f in ohlcs], axis=0, sort=True)
df_und = pd.concat([pd.read_pickle(fspath+f) for f in unds])

# remove options in black list
df_opt = df_opt[~df_opt.symbol.isin(blacklist)]

# arrange the columns
cols = ['symbol', 'right', 'expiry', 'dte', 'strike', 'undPrice', 'lo52',  'hi52', 
'stdev', 'volatility', 'margin', 'lot', 'bsmPrice', 'pop', 'rom', 'price', 'option']

df_opt1 = df_opt[cols]

# take only high pops
df_opt2 = df_opt1[df_opt1['pop'] >= min_pop].reset_index(drop=True)

# take only top stocks
# df_opt2 = df_opt2[df_opt2.symbol.isin(topstocks)]

# take only puts
df_opt3 = get_onlyputs(df_opt2)

# keep only options with prices
df_opt3 = df_opt3[df_opt3.price > 0]

df_opt3 = df_opt3.sort_values('rom', ascending = False)  

df_opt3 = df_opt3[df_opt3.rom > min_rom] # Limit above minimum rom

In [ ]:
df_opt4 = df_opt3.assign(margin=[get_opt_margin(ib, o, l) for o, l in zip(df_opt3.option, itertools.repeat(1))])

df_opt4 = df_opt4[df_opt4.margin < df_opt4.strike*df_opt4.lot] # remove 1.7976931348623157e+308 from margins!

# get the better of Price and bsmPrice for the option - with an expected multiple (expmult)
max_price_bsm = pd.concat([df_opt4.price, df_opt4.bsmPrice], axis=1).max(axis=1)
df_opt4['expPrice'] = get_prec(max_price_bsm*expmult, base)

df_opt4 = df_opt4.assign(rom=df_opt4.expPrice*df_opt4.lot/df_opt4.margin*365/df_opt4.dte).reset_index(drop=True).sort_values('rom', ascending=False)

In [ ]:
# Make df the dataframe that you want to execute on!
df = df_opt4.copy()   # make this the last dataframe to get the orders placed

df.loc[df.expPrice < 0.2, 'expPrice'] = 0.2  # Make the selling price a minimum of 0.2

contracts = [c for c in df.option]

df = df.assign(qty=pd.concat([round(ass_limit / (df.strike * df.lot)), 
                         round(max_p/df.margin)], axis=1).min(axis=1))

# df = df.assign(qty=1) # defaults to a quantity of 1
    
print('{:d} contracts from {:d} scrips, consuming {:,.0f} margin from full available funds of {:,.0f}, giving USD {:,.0f}'.format(len(contracts), \
      len(df.symbol.unique()), sum(df.margin*df.qty), av_funds*max_nlvp, sum(df.expPrice*df.lot*df.qty)))

In [ ]:
# ...review calls and puts
# e.g. for puts from path: C:\Users\kashir\Documents\IBKR\snp\zdata\putswatch.csv
# .....,, or in this path: C:\Users\User\Documents\ibkr\snp\zdata\putswatch.csv (home laptop)

# add tgtPrice based on desired rom
df['tgtPrice'] = get_prec(pd.concat([df.rom, pd.Series(desired_rom, index=df.index)], axis=1).max(axis=1)*df.expPrice/df.rom, base)

df['remqty'] = round(max_p/df.margin)

# replace remqty with non-blacklist remaining quantities
if ps:
    df = df.set_index('symbol')
    df.remqty = np.where(df.index.isin(remqtydict.keys()), df.index.map(remqtydict), df.remqty) 
    df = df.reset_index()

# Sort the calls and puts by symbol and strikes - to quickly weed out risky options
df = grp_opts(df)

cols = ['right', 'symbol', 'strike', 'undPrice', 'dte', 'pop', 'rom', 'price', 'expPrice', 'margin', 'lot', 'qty', 'remqty', 'tgtPrice', 'option']
df = df[cols]

# make watchlists
df_puts = df[df.right == 'P'].reset_index(drop=True)
df_calls = df[df.right == 'C'].reset_index(drop=True)

if not df_puts.empty:
    watchputs = [('DES', s, 'STK', 'SMART/ARCA') for s in df_puts.symbol.unique()]
    df_wp = util.df(watchputs)
    df_wp.to_csv(fspath+'putswatch.csv', index=None, header=False)

if not df_calls.empty:
    watchcalls = [('DES', s, 'STK', 'SMART/ARCA') for s in df_calls.symbol.unique()]
    df_wc = util.df(watchcalls)
    df_wc.to_csv(fspath+'.callswatch.csv', index=None, header=False)

df.to_csv(fspath+'check.csv', index=None, header=True)

In [ ]:
# .......for puts from path: C:\Users\kashir\Documents\IBKR\data\snp\putswatch.csv
# ....... or in this path: C:\Users\User\Documents\ibkr\data\snp\putswatch.csv (home laptop)
# After going through checked.csv, with puts and calls, eliminate risky options
# Save the file as checked.csv

df_final = pd.read_csv(fspath+'checked.csv') # picks up the checked and ready-to-go contracts
cs = [eval(c) for c in df_final.option]  # convert the "quoted strings" from csv back to object
orders = [LimitOrder(action='SELL', totalQuantity=qty, lmtPrice=tgtPrice) for qty, tgtPrice in zip(df_final.qty, df_final.tgtPrice)]
print('{:d} contracts from {:d} scrips, consuming {:,.0f} margin from full available funds of {:,.0f}, giving USD {:,.0f} in premiums.'.format(len(df_final), \
      len(df_final.symbol.unique()), sum(df_final.margin*df_final.qty), av_funds*max_nlvp, sum(df_final.tgtPrice*df_final.qty*100)))

In [ ]:
# look at the amount of margin consumed
df_final.assign(totmargin = df_final.margin * df_final.qty).groupby('symbol', as_index=False).agg({'totmargin' : 'sum'}).sort_values('totmargin', ascending=False).reset_index(drop=True)